In [19]:
import cv2
import numpy as np
import face_detection
import collections
import os, python_speech_features
import scipy.io.wavfile as wav
import random
import pandas as pd
import pickle
import subprocess
import torch
from talkNet import talkNet
from dataset import MyDataset
detector = face_detection.build_detector(
  "DSFDDetector", confidence_threshold=.2, nms_iou_threshold=.2)

c:\Users\jmmol\miniconda3\envs\alc\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jmmol\miniconda3\envs\alc\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [20]:
inputVideo =  "C:/Users/jmmol/Desktop/LIP-RTVE/MP4s/speaker042/speaker042_0007.mp4"

def convert_video_to_audio_ffmpeg(video_file, output_ext="wav"):
    """Extracts audio from video to .wav format, returns path of the resulting audio"""
    filename, ext = os.path.splitext(video_file)
    filename = filename.split('/')[-1]
    #Gets working directory and extracts audio
    subprocess.call(["ffmpeg", "-y", "-i", video_file, os.getcwd()+f"/{filename}.{output_ext}"],
                    stdout=subprocess.DEVNULL,
                    stderr=subprocess.STDOUT)
    return os.getcwd()+f"/{filename}.{output_ext}" #Nombre del audio

def extractBiggestFace(img):
    """
    Detecta todas las caras de una imagen y devuelve la más grande recortada y reescalada a 112x112
    """
    detections = detector.detect(img)
    idx_max = -1
    area_max = -1
    for i,cntr in enumerate(detections):
        xmin,ymin,xmax,ymax = int(cntr[0]),int(cntr[1]),int(cntr[2]),int(cntr[3]) #Guardamos bounding box
        area = (xmax-xmin)*(ymax-ymin)
        if area > area_max: #Comprobamos si la cara es la más grande
            idx_max = i
            area_max = area
            #print(area,idx_max)
        #cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)

    cntr = detections[idx_max]
    try:
        xmin,ymin,xmax,ymax = int(cntr[0]),int(cntr[1]),int(cntr[2]),int(cntr[3])
        resImage = cv2.resize(img[max(ymin,0):ymax, xmin:xmax], (112, 112)) #Cara detectada, reescalamos
        resImage = cv2.cvtColor(resImage, cv2.COLOR_BGR2GRAY)
        return resImage
    except:
        print(cntr)
        cv2.imshow('image',img)
        cv2.waitKey(0)

def saveFaceCrops(videoPath):
    vidcap = cv2.VideoCapture(videoPath)
    success,image = vidcap.read()
    count = 0
    faceArray = []
    while success:
        faceArray.append(extractBiggestFace(image)) #DE MOMENTO SIEMPRE HAY CARA
        success,image = vidcap.read()
        count += 1
    return faceArray #Devuelve número de frames



In [21]:
res = saveFaceCrops(inputVideo)
res[0].shape

# cv2.imshow('image',res[3])
# cv2.waitKey(0)

(112, 112)

In [22]:
# AUDIO PROCESSING
audioPath = convert_video_to_audio_ffmpeg(inputVideo)
_,sig = wav.read(audioPath)#r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker042_0063.wav") #
_,sig = wav.read(r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker042_0004.wav") 
audio = python_speech_features.mfcc(sig, 16000, numcep = 13, winlen = 0.025, winstep = 0.010) #ASUME VIDEO A 25 Y AUDIO A 100, MODIFICAR


#TEST
audio = torch.FloatTensor(audio[:28]).unsqueeze(0)
video = torch.FloatTensor(res[:7]).unsqueeze(0)
print("audio shape: ", audio.shape)
print("video shape: ", video.shape)


audio shape:  torch.Size([1, 28, 13])
video shape:  torch.Size([1, 7, 112, 112])


In [23]:
videoDir = "C:/Users/jmmol/Desktop/COSAS V7/TFM/npz"
audioDir = "C:/Users/jmmol/Desktop/COSAS V7/TFM/mfccs"
datasetTrain = MyDataset(51,videoDir,audioDir,"trainSamples.csv")
item = datasetTrain.__getitem__(1)

25 100


In [24]:
model = talkNet()
model.load_state_dict(torch.load("./exps/exp2/model/model_0012.model"))

05-19 19:11:35 Model para number = 15.01


<All keys matched successfully>

In [25]:
#output = model((item[0].unsqueeze(0),item[1].unsqueeze(0)))
output = model((audio,video))

torch.Size([1, 28, 13]) torch.Size([1, 7, 112, 112])


In [26]:
output

(tensor(0.2311, device='cuda:0'),
 tensor([[0.2008, 0.7992],
         [0.2021, 0.7979],
         [0.2508, 0.7492],
         [0.1506, 0.8494],
         [0.1983, 0.8017],
         [0.2033, 0.7967],
         [0.2349, 0.7651]], device='cuda:0'),
 tensor([1., 1., 1., 1., 1., 1., 1.], device='cuda:0'),
 tensor(7., device='cuda:0'))

In [27]:
torch.argmax(output, dim=1)

TypeError: argmax(): argument 'input' (position 1) must be Tensor, not tuple